<a href="https://colab.research.google.com/github/AminHasibul/Continual-Learning-/blob/main/Solving_Catastrophic_fogetting_for_continual_learning_using_custom_ResNET_and_ICF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Core framework

In [ ]:
!pip install avalanche-lib --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.2/134.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.4/993.4 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 186.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.2/585.2 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.9/981.9 kB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 129.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.7/356.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 158.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



# Datasets & transforms

In [ ]:

from torch.utils.data import DataLoader, Subset
import torchvision.datasets as datasets
import torchvision.transforms as transforms



# Continual‐learning library

In [ ]:
print("Avalanche version:", __import__('avalanche').__version__)

Avalanche version: 0.6.0


In [ ]:

from avalanche.benchmarks import SplitCIFAR10, nc_benchmark
from avalanche.training.supervised import Naive, EWC
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import accuracy_metrics, forgetting_metrics



# Clustering for RICS

In [ ]:

from sklearn.cluster import KMeans
import numpy as np

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define MLP Model

A two‑layer CNN with pooling, then a 256‑unit dense layer. return_feats lets us extract the feature vector for RICS clustering.

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden=256, total_classes=10):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, total_classes)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Define CNN Model

A two‑layer CNN with pooling, then a 256‑unit dense layer. return_feats lets us extract the feature vector for RICS clustering.

In [ ]:
# Updated CNN that adapts to image size
class simpleCNN(nn.Module):
    def __init__(self, in_ch, num_classes, img_size):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool  = nn.MaxPool2d(2, 2)
        # After two pools, spatial dims = img_size / 4
        conv_out = img_size // 4
        self.fc1 = nn.Linear(64 * conv_out * conv_out, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x, return_feats=False):
        x = F.relu(self.conv1(x)); x = self.pool(x)
        x = F.relu(self.conv2(x)); x = self.pool(x)
        feats = x.view(x.size(0), -1)
        out   = self.fc2(F.relu(self.fc1(feats)))
        return (out, feats) if return_feats else out



#RICS: Centroid Computation & Loss

After each task we store class centroids in the model’s feature space. During the next task, we recluster new features and penalize the shift of each centroid—this is the RICS regularizer.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from avalanche.training.templates import SupervisedTemplate
from typing import Dict

In [ ]:


# ----------------------------
# Simple CNN Model
# ----------------------------
class SimpleCNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=10, feat_dim=256):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64 * 8 * 8, feat_dim)
        self.fc2 = nn.Linear(feat_dim, num_classes)

    def forward(self, x, return_feats=False):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        feats = F.relu(self.fc1(x))
        logits = self.fc2(feats)
        return (feats, logits) if return_feats else logits

#  Wrap RICS as an Avalanche Strategy

You’ll subclass Avalanche’s BaseStrategy to inject the RICS penalty into the training loop. Below is a sketch—you’ll fill in the details:

SupervisedTemplate gives you hooks like before_training_exp, after_training_exp, and criterion to insert your logic.
In after_training_exp, we compute and store the centroids of the just-learned task.
In criterion, we add the RICS penalty using those stored centroids.

In [ ]:
from avalanche.training.templates import SupervisedTemplate

#  Define Benchmarks

Explanation:

Class‑IL tests completely new labels.

Domain‑IL tests same labels under input shifts.

Similar‑Class tests fine‑grained, semantically related classes.



A. Class‑IL on CIFAR‑10 (Disjoint)

In [ ]:
benchmark_class_il = SplitCIFAR10(
    n_experiences=5,
    return_task_id= False,
    fixed_class_order=list(range(10))
)


100%|██████████| 170M/170M [00:09<00:00, 17.5MB/s]


In [ ]:
print("Class‑IL experiences:", len(benchmark_class_il.train_stream))

Class‑IL experiences: 2


B. Domain‑IL on CIFAR‑10 (Permuted)

In [ ]:
import numpy as np



Create two pixel‑permuted variants


# Download base

In [ ]:

base_train = datasets.CIFAR10(root='./data', train=True, download=True,
                              transform=transforms.ToTensor())
base_test  = datasets.CIFAR10(root='./data', train=False,download=True,
                              transform=transforms.ToTensor())

In [ ]:
from torchvision.datasets import CIFAR10
from avalanche.benchmarks import nc_benchmark

In [ ]:
perms = [np.random.permutation(32 * 32 * 3) for _ in range(2)]

In [ ]:
# Helper: permuted dataset
class PermutedCIFAR10(CIFAR10):
    def __init__(self, base_ds, perm):
        super().__init__(base_ds.root, train=base_ds.train, download=False, transform=base_ds.transform)
        flat = base_ds.data.reshape(-1, 32*32*3)
        self.data = flat[:, perm].reshape(-1, 32, 32, 3)
        self.targets = base_ds.targets



train_sets = [PermutedCIFAR10(base_train, p) for p in perms]
test_sets  = [PermutedCIFAR10(base_test,  p) for p in perms]

# Create Avalanche benchmark (no task_names parameter)
benchmark_domain_il = nc_benchmark(
    train_sets,
    test_sets,
    2,
    task_labels=[0, 1],        # two tasks
    shuffle=False
)



In [ ]:
print("Domain‑IL experiences:", len(benchmark_domain_il.train_stream))

Domain‑IL experiences: 2


C.  Similar‑Class Variation (Vehicle Subclasses)

In [ ]:
from torch.utils.data import Subset


Vehicle classes: 0=airplane,1=automobile,8=ship,9=truck

In [ ]:

group1, group2 = [0,1], [8,9]

def subset_by_classes(ds, classes):
    idx = [i for i,y in enumerate(ds.targets) if y in classes]
    return Subset(ds, idx)

train_sets = [subset_by_classes(base_train, group1),
              subset_by_classes(base_train, group2)]
test_sets  = [subset_by_classes(base_test,  group1),
              subset_by_classes(base_test,  group2)]

benchmark_similar = nc_benchmark(
    train_sets,
    test_sets,
    2,
    task_labels=[0,1],
    shuffle=False)


In [ ]:
print("Similar‑Class IL experiences:", len(benchmark_similar.train_stream))

Similar‑Class IL experiences: 2


# Run Strategies & Collect Results

This loop runs all 3 strategies on each of the 3 scenarios; Avalanche handles the “train then test on all seen tasks” logic and collects accuracy/forgetting metrics automatically.

In [ ]:
device = "cpu"


Common evaluation plugin

Run Strategies on All Scenarios


In [ ]:
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics, forgetting_metrics, timing_metrics
from avalanche.logging import InteractiveLogger
from avalanche.training.plugins import EvaluationPlugin

# Define logger
logger = InteractiveLogger()

# Create evaluation plugin
eval_plugin = EvaluationPlugin(
    accuracy_metrics(epoch=True, experience=True, stream=True),
    loss_metrics(epoch=True, experience=True, stream=True),
    loggers=[logger]
)

# Scenarios Class‑IL

In [ ]:
bench = benchmark_class_il
in_ch = 3

print(f"\n=== Scenario: Class‑IL (CIFAR‑10) ===")

print(f"\n-- Model: CNN | Strategy: Naive --")


=== Scenario: Class‑IL (CIFAR‑10) ===

-- Model: CNN | Strategy: Naive --


Naive

In [ ]:

strategy = Naive(**common)

# train & eval

  for experience in bench.train_stream:
      current_classes = list(set(experience.dataset.targets))
      print(f"Training on classes: {current_classes}")
      print("Training on experience", experience.current_experience)

      strategy.train(experience)

      print("Evaluating on all seen tasks...")
      strategy.eval(bench.test_stream[: experience.current_experience + 1])


EWC

In [ ]:

strategy = EWC(**common, ewc_lambda=0.5)

print(f"\n-- Model: CNN | Strategy: EWC --")

# train & eval

  for experience in bench.train_stream:
      current_classes = list(set(experience.dataset.targets))
      print(f"Training on classes: {current_classes}")
      print("Training on experience", experience.current_experience)

      strategy.train(experience)

      print("Evaluating on all seen tasks...")
      strategy.eval(bench.test_stream[: experience.current_experience + 1])

Our Model RICS

In [ ]:
strategy = RICSStrategy(**common, ewc_lambda=0.5)

print(f"\n-- Model: CNN | Strategy: RICS --")

# train & eval

  for experience in bench.train_stream:
      current_classes = list(set(experience.dataset.targets))
      print(f"Training on classes: {current_classes}")
      print("Training on experience", experience.current_experience)

      strategy.train(experience)

      print("Evaluating on all seen tasks...")
      strategy.eval(bench.test_stream[: experience.current_experience + 1])

In [ ]:
print("Sample labels in this task:", set(exp.dataset.targets))

NameError: name 'exp' is not defined

#Scenario Domain IL

In [ ]:
bench = benchmark_domain_il
bench = benchmark_class_il
in_ch = 3

print(f"\n=== Scenario: Domain‑IL (CIFAR‑10) ===")

print(f"\n-- Model: CNN | Strategy: Naive --")

Naive

In [ ]:

strategy = Naive(**common)

# train & eval

  for experience in bench.train_stream:
      current_classes = list(set(experience.dataset.targets))
      print(f"Training on classes: {current_classes}")
      print("Training on experience", experience.current_experience)

      strategy.train(experience)

      print("Evaluating on all seen tasks...")
      strategy.eval(bench.test_stream[: experience.current_experience + 1])


EWC

In [ ]:

strategy = EWC(**common, ewc_lambda=0.5)

print(f"\n-- Model: CNN | Strategy: EWC --")

# train & eval

  for experience in bench.train_stream:
      current_classes = list(set(experience.dataset.targets))
      print(f"Training on classes: {current_classes}")
      print("Training on experience", experience.current_experience)

      strategy.train(experience)

      print("Evaluating on all seen tasks...")
      strategy.eval(bench.test_stream[: experience.current_experience + 1])

RICS

In [ ]:
strategy = RICSStrategy(**common, ewc_lambda=0.5)

print(f"\n-- Model: CNN | Strategy: RICS --")

# train & eval

  for experience in bench.train_stream:
      current_classes = list(set(experience.dataset.targets))
      print(f"Training on classes: {current_classes}")
      print("Training on experience", experience.current_experience)

      strategy.train(experience)

      print("Evaluating on all seen tasks...")
      strategy.eval(bench.test_stream[: experience.current_experience + 1])

In [ ]:
cifar_scenarios = [
    ("Class‑IL",   benchmark_class_il)
]

strategies = [("Naive", Naive), ("EWC", EWC), ("RICS", RICSStrategy)]

for scen_name, bench in cifar_scenarios:
    print(f"\n=== Scenario: {scen_name} (CIFAR‑10) ===")

    for strat_name, Strat in strategies:
        print(f"\n-- Model: CNN | Strategy: {strat_name} --")

        for experience in bench.train_stream:
            current_classes = experience.classes_in_this_experience
            num_classes = len(set(experience.dataset.targets))
            print(f"Training on classes: {experience.dataset.targets}")

            # Step 4: Recreate model with matching output size
            model = SimpleCNN(in_channels=3, num_classes=num_classes).to(device)
            optimizer = optim.Adam(model.parameters(), lr=0.001)
            criterion = nn.CrossEntropyLoss()

            # === Strategy creation ===
            if strat_name == "EWC":
                strategy = Strat(
                    model=model,
                    optimizer=optimizer,
                    criterion=criterion,
                    ewc_lambda=0.5,
                    train_mb_size=64,
                    train_epochs=2,
                    eval_mb_size=64,
                    device=device,
                    evaluator=eval_plugin
                )
            elif strat_name == "RICS":
                strategy = Strat(
                    model=model,
                    optimizer=optimizer,
                    criterion=criterion,
                    rics_lambda=0.5,
                    train_mb_size=64,
                    train_epochs=2,
                    eval_mb_size=64,
                    device=device,
                    evaluator=eval_plugin
                )
            else:  # Naive
                strategy = Strat(
                    model=model,
                    optimizer=optimizer,
                    criterion=criterion,
                    train_mb_size=64,
                    train_epochs=2,
                    eval_mb_size=64,
                    device=device,
                    evaluator=eval_plugin
                )
            print("Training on experience", experience.current_experience)
            print("Model output dim:", model.fc1.out_features)


            # === Train & Evaluate ===
            print(f"\nTraining on Task {experience.current_experience}")
            strategy.train(experience)

            print("Evaluating on all seen tasks...")
            print("Evaluating on all seen tasks...")
            # Evaluate on all test experiences so far
            test_experiences = (bench.test_stream)[: experience.current_experience + 1]
            strategy.eval(test_experiences)



=== Scenario: Class‑IL (CIFAR‑10) ===

-- Model: CNN | Strategy: Naive --
Training on classes: [0, 1, 2, 3, 4]
Training on experience 0
Model output dim: 256

Training on Task 0
-- >> Start of training phase << --
100%|██████████| 391/391 [00:08<00:00, 47.80it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.0405
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5738
100%|██████████| 391/391 [00:07<00:00, 51.02it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.8256
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6692
-- >> End of training phase << --
Evaluating on all seen tasks...
Evaluating on all seen tasks...
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 92.56it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.6889
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7356
-- >> End of eval

IndexError: Target 6 is out of bounds.

ImportError: cannot import name 'AvalancheDataset' from 'avalanche.benchmarks.utils.dataset_utils' (/usr/local/lib/python3.11/dist-packages/avalanche/benchmarks/utils/dataset_utils.py)

In [ ]:
cifar_scenarios = [
    ("Class‑IL",   benchmark_class_il)
]

strategies = [("Naive", Naive), ("EWC", EWC), ("RICS", RICSStrategy)]

for scen_name, bench in cifar_scenarios:
    print(f"\n=== Scenario: {scen_name} (CIFAR‑10) ===")

    for strat_name, Strat in strategies:
        print(f"\n-- Model: CNN | Strategy: {strat_name} --")

        for experience in bench.train_stream:
            current_classes = list(set(experience.dataset.targets))
            num_classes = max(current_classes) + 1

            print(f"Training on classes: {current_classes}")

            # Step 4: Recreate model with matching output size
            model = SimpleCNN(in_channels=3, num_classes=num_classes).to(device)
            optimizer = optim.Adam(model.parameters(), lr=0.001)
            criterion = nn.CrossEntropyLoss()

            # === Strategy creation ===
            if strat_name == "EWC":
                strategy = Strat(
                    model=model,
                    optimizer=optimizer,
                    criterion=criterion,
                    ewc_lambda=0.5,
                    train_mb_size=64,
                    train_epochs=2,
                    eval_mb_size=64,
                    device=device,
                    evaluator=eval_plugin
                )
            elif strat_name == "RICS":
                strategy = Strat(
                    model=model,
                    optimizer=optimizer,
                    criterion=criterion,
                    rics_lambda=0.5,
                    train_mb_size=64,
                    train_epochs=2,
                    eval_mb_size=64,
                    device=device,
                    evaluator=eval_plugin
                )
            else:  # Naive
                strategy = Strat(
                    model=model,
                    optimizer=optimizer,
                    criterion=criterion,
                    train_mb_size=64,
                    train_epochs=2,
                    eval_mb_size=64,
                    device=device,
                    evaluator=eval_plugin
                )
            print("Training on experience", experience.current_experience)
            print("Model output dim:", model.fc1.out_features)


            # === Train & Evaluate ===
            print(f"\nTraining on Task {experience.current_experience}")
            strategy.train(experience)

            print("Evaluating on all seen tasks...")
            print("Evaluating on all seen tasks...")
            # Evaluate on all test experiences so far
            test_experiences = list(bench.test_stream)[: experience.current_experience + 1]
            strategy.eval(test_experiences)



=== Scenario: Class‑IL (CIFAR‑10) ===

-- Model: CNN | Strategy: Naive --
Training on classes: [0, 1, 2, 3, 4]
Training on experience 0
Model output dim: 256

Training on Task 0
-- >> Start of training phase << --
100%|██████████| 391/391 [11:23<00:00,  1.75s/it]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.0527
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5704
100%|██████████| 391/391 [00:07<00:00, 49.27it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.8332
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6726
-- >> End of training phase << --
Evaluating on all seen tasks...
Evaluating on all seen tasks...
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 89.28it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.6892
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7334
-- >> End of eval

/usr/local/lib/python3.11/dist-packages/avalanche/training/templates/base.py:389: UserWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your strategy RICSStrategy.__init__ method still has some positional-only or positional-or-keyword arguments. Consider removing them. Offending arguments: ['self', 'model', 'optimizer', 'criterion', 'rics_lambda', 'train_mb_size', 'train_epochs', 'eval_mb_size', 'device', 'evaluator']. This can be achieved by adding a * in the argument list of your __init__ method just after 'self'. More info: https://peps.python.org/pep-3102/#specification
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/avalanche/training/templates/base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This

Training on experience 0
Model output dim: 256

Training on Task 0
-- >> Start of training phase << --
100%|██████████| 391/391 [00:07<00:00, 51.24it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.1118
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5479
100%|██████████| 391/391 [00:08<00:00, 47.92it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.8540
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6620
-- >> End of training phase << --
Evaluating on all seen tasks...
Evaluating on all seen tasks...
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 86.54it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7308
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7098
-- >> End of eval phase << --
	Loss_Stream/eval_phase/test_stream/Task000 = 0.7308
	Top1_Acc_Stream/eval_phase/test_stream/Task00

In [ ]:
cifar_scenarios = [
    ("Class‑IL",   benchmark_class_il,   3, 32)
]

strategies = [("Naive", Naive), ("EWC", EWC), ("RICS", RICSStrategy)]

for scen_name, bench, in_ch, img_size in cifar_scenarios:
    print(f"\n=== Scenario: {scen_name} (CIFAR‑10) ===")

    for strat_name, Strat in strategies:
        print(f"\n-- Model: CNN | Strategy: {strat_name} --")

        model = SimpleCNN(in_channels=3, num_classes=10).to(device)
        optimizer = optim.Adam(model.parameters(), lr=1e-3)

        common = dict(
            model=model, optimizer=optimizer, criterion=nn.CrossEntropyLoss(),
            train_mb_size=64, train_epochs=5,
            eval_mb_size=64,
            device=device, evaluator=eval_plugin
        )

        if strat_name == "EWC":
            strategy = Strat(**common, ewc_lambda=0.5)
        elif strat_name == "RICS":
            strategy = Strat(**common, rics_lambda=0.5)
        else:
            strategy = Strat(**common)

        # train & eval

        for experience in bench.train_stream:
            current_classes = list(set(experience.dataset.targets))
            print(f"Training on classes: {current_classes}")
            print("Training on experience", experience.current_experience)

            strategy.train(experience)

            print("Evaluating on all seen tasks...")
            strategy.eval(bench.test_stream[: experience.current_experience + 1])



=== Scenario: Class‑IL (CIFAR‑10) ===

-- Model: CNN | Strategy: Naive --
Training on classes: [0, 1]
Training on experience 0
-- >> Start of training phase << --
100%|██████████| 157/157 [00:03<00:00, 49.74it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4600
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7899
100%|██████████| 157/157 [00:03<00:00, 49.17it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2988
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8760
100%|██████████| 157/157 [00:03<00:00, 49.46it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2529
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8982
100%|██████████| 157/157 [00:03<00:00, 49.28it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2110
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9151
100%|██████████| 157/157 [00:03<00:00, 48.50it/s]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1941
	Top1_Acc

/usr/local/lib/python3.11/dist-packages/avalanche/training/templates/base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the SupervisedTemplate.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


Training on classes: [0, 1]
Training on experience 0
-- >> Start of training phase << --
100%|██████████| 157/157 [00:03<00:00, 49.31it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4496
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7899
100%|██████████| 157/157 [00:03<00:00, 48.83it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2894
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8733
100%|██████████| 157/157 [00:03<00:00, 48.32it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2389
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9004
100%|██████████| 157/157 [00:03<00:00, 49.42it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2096
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9135
100%|██████████| 157/157 [00:03<00:00, 49.36it/s]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1837
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9278
-- >> End of training phas

## Approch 2

# rics_cnn_experiment.py
"""End‑to‑end reproducible experiment script for CIFAR‑10 Class‑IL.

It runs three strategies side‑by‑side:
* Naïve fine‑tuning (baseline)
* EWC       (elastic‑weight consolidation)
* RICS      (our proposed method)

Requirements (tested with Avalanche 0.4.0):
    pip install avalanche-lib torch torchvision
"""

In [ ]:
class ClusterAwareReplayFreeCL(SupervisedTemplate):
    def __init__(self, *, model, optimizer, lambda_intra=1.0, lambda_anchor=10.0, lambda_logit=1.0, lambda_var=1.0, criterion=nn.CrossEntropyLoss(), **kwargs):
        super().__init__(model=model, optimizer=optimizer, criterion=criterion, **kwargs)

        # Save base values for decay
        self.base_lambda_anchor = lambda_anchor
        self.base_lambda_logit = lambda_logit
        self.base_lambda_var = lambda_var

        self.lambda_intra = lambda_intra
        self.lambda_anchor = lambda_anchor
        self.lambda_logit = lambda_logit
        self.lambda_var = lambda_var
        self.prev_centroids: Dict[int, torch.Tensor] = {}
        self.prev_variances: Dict[int, torch.Tensor] = {}
        self.model_old = None
        self.logits = None
        self.current_feats = None

    def forward(self):
        self.current_feats, self.logits = self.model(self.mb_x, return_feats=True)
        return self.logits

    def criterion(self):
        ce_loss = self._criterion(self.logits, self.mb_y)
        feats = F.normalize(self.current_feats, p=2, dim=1)
        targets = self.mb_y

        # Current centroids and variances
        curr_centroids, curr_variances = {}, {}
        for cls in torch.unique(targets):
          mask = (targets == cls)
          feats_cls = F.normalize(feats[mask], p=2, dim=1)  # Normalize feature vectors
          curr_centroids[cls.item()] = feats_cls.mean(0)
          curr_variances[cls.item()] = feats_cls.var(0)

        # Intra-task cluster separation
        inter_cluster_loss = 0.0
        cls_list = list(curr_centroids.keys())
        for i in range(len(cls_list)):
            for j in range(i + 1, len(cls_list)):
                inter_cluster_loss += F.mse_loss(curr_centroids[cls_list[i]], curr_centroids[cls_list[j]])
        if len(cls_list) > 1:
            inter_cluster_loss /= len(cls_list) * (len(cls_list) - 1) / 2

        # Cross-task centroid anchoring
        anchor_loss = 0.0
        for cls, prev_mu in self.prev_centroids.items():
            if cls in curr_centroids:
                anchor_loss += F.mse_loss(curr_centroids[cls], prev_mu.to(feats.device))
        if self.prev_centroids:
            anchor_loss /= len(self.prev_centroids)

        # Variance anchoring
        var_loss = 0.0
        for cls, prev_var in self.prev_variances.items():
            if cls in curr_variances:
                var_loss += F.mse_loss(curr_variances[cls], prev_var.to(feats.device))
        if self.prev_variances:
            var_loss /= len(self.prev_variances)

        logit_loss = 0.0
        if self.model_old is not None:
            with torch.no_grad():
                old_logits = self.model_old(self.mb_x)
            logit_loss = F.mse_loss(self.logits, old_logits)

        #logit_loss = 0.0
        # if self.model_old is not None:
        #     with torch.no_grad():
        #         old_logits = self.model_old(self.mb_x)
        #     logit_loss = 1 - F.cosine_similarity(self.logits, old_logits, dim=1).mean()

        # norm_loss = 0.0
        # if self.model_old is not None:
        #     for (p_new, p_old) in zip(self.model.parameters(), self.model_old.parameters()):
        #         norm_loss += torch.norm(p_new - p_old, p=2)

        feat_loss = 0.0
        if self.model_old is not None:
            with torch.no_grad():
                old_feats, _ = self.model_old(self.mb_x, return_feats=True)
            feat_loss = F.mse_loss(self.current_feats, old_feats)

        return (
            ce_loss +
            self.lambda_intra * inter_cluster_loss +
            self.lambda_anchor * anchor_loss +
            self.lambda_logit * logit_loss +
            self.lambda_var * var_loss +
            0.001 * feat_loss  # small weight to avoid hurting plasticity
        )


        # return (
        #     ce_loss +
        #     self.lambda_intra * inter_cluster_loss +
        #     self.lambda_anchor * anchor_loss +
        #     self.lambda_logit * logit_loss +
        #     self.lambda_var * var_loss
        #)

    def _before_training_iteration(self, **kwargs):
        self.logits = None
        self.current_feats = None
        current_task = self.experience.current_experience
        # Adaptive decay
        decay_factor = 0.5 ** current_task
        self.lambda_anchor = self.base_lambda_anchor * decay_factor
        self.lambda_logit  = self.base_lambda_logit * decay_factor
        self.lambda_var    = self.base_lambda_var * decay_factor

        #print(f"[λ] Task {current_task} | λ_anchor={self.lambda_anchor:.3f} | λ_logit={self.lambda_logit:.3f} | λ_var={self.lambda_var:.3f}")

        super()._before_training_iteration(**kwargs)

    def _after_training_exp(self, **kwargs):
        print("[Replay-Free CL] Saving centroids, variances, and freezing model.")
        self.model.eval()
        dl = DataLoader(self.experience.dataset.eval(), batch_size=256, shuffle=False)
        centroids, variances = {}, {}

        with torch.no_grad():
            for x, y, *_ in dl:
                x, y = x.to(self.device), y.to(self.device)
                feats, _ = self.model(x, return_feats=True)
                feats = F.normalize(feats, p=2, dim=1)
                for cls in torch.unique(y):
                    mask = y == cls
                    feats_cls = feats[mask]
                    if mask.any():
                        if cls.item() not in centroids:
                            centroids[cls.item()] = []
                            variances[cls.item()] = []
                        centroids[cls.item()].append(feats_cls.mean(0))
                        variances[cls.item()].append(feats_cls.var(0))

        for cls in centroids:
            self.prev_centroids[cls] = torch.stack(centroids[cls]).mean(0).detach().cpu()
            self.prev_variances[cls] = torch.stack(variances[cls]).mean(0).detach().cpu()


        # Freeze previous model for logit anchoring
        self.model_old = SimpleCNN()
        self.model_old.load_state_dict(self.model.state_dict())
        self.model_old.to(self.device)
        self.model_old.eval()

        print(f"[Freeze] Model frozen at task {self.experience.current_experience}")

        for p in self.model_old.parameters():
            p.requires_grad = False

        self.model.train()
        super()._after_training_exp(**kwargs)


Reply based


In [ ]:
model = SimpleCNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

strategy = ClusterAwareReplayFreeCL(
    model=model,
    optimizer=optimizer,
    train_mb_size=64,
    train_epochs=5,
    eval_mb_size=128,
    device=device,
    lambda_intra=5.0,
    lambda_logit= 1,
    lambda_anchor = 0.5,
    lambda_var =0,
    evaluator=eval_plugin
)


In [ ]:
# train & eval

for experience in benchmark_class_il.train_stream:
    current_classes = list(set(experience.dataset.targets))
    print(f"Training on classes: {current_classes}")
    print("Training on experience", experience.current_experience)

    strategy.train(experience)

    print("Evaluating on all seen tasks...")
    strategy.eval(benchmark_class_il.test_stream[: experience.current_experience + 1])

Training on classes: [0, 1]
Training on experience 0
-- >> Start of training phase << --
100%|██████████| 157/157 [03:38<00:00,  1.39s/it]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4610
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7812
100%|██████████| 157/157 [00:03<00:00, 48.92it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2887
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8800
100%|██████████| 157/157 [00:03<00:00, 44.29it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2354
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9026
100%|██████████| 157/157 [00:03<00:00, 49.04it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2159
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9134
100%|██████████| 157/157 [00:03<00:00, 48.11it/s]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1959
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9213
[Replay-Free CL] Saving ce

In [ ]:
from torchvision.models import resnet18

model = resnet18(pretrained=False, num_classes=10)
model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = torch.nn.Identity()  # remove downsampling for CIFAR

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
# train & eval
strategy = ClusterAwareReplayFreeCL(
    model=model,
    optimizer=optimizer,
    train_mb_size=64,
    train_epochs=5,
    eval_mb_size=128,
    device=device,
    lambda_intra= 5,
    lambda_logit= 0.5,
    lambda_anchor = 0.5,
    lambda_var = 0,
    evaluator=eval_plugin
)


for experience in benchmark_class_il.train_stream:
    current_classes = list(set(experience.dataset.targets))
    print(f"Training on classes: {current_classes}")
    print("Training on experience", experience.current_experience)

    strategy.train(experience)

    print("Evaluating on all seen tasks...")
    strategy.eval(benchmark_class_il.test_stream[: experience.current_experience + 1])

Training on classes: [0, 1]
Training on experience 0
-- >> Start of training phase << --
0it [00:00, ?it/s]

TypeError: ResNet.forward() got an unexpected keyword argument 'return_feats'

In [ ]:
strategy.train(benchmark_class_il.train_stream)
strategy.eval(benchmark_class_il.test_stream)


-- >> Start of training phase << --
100%|██████████| 157/157 [05:32<00:00,  2.12s/it]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4369
	Time_Epoch/train_phase/train_stream/Task000 = 3.1700
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7977
100%|██████████| 157/157 [00:03<00:00, 52.16it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2780
	Time_Epoch/train_phase/train_stream/Task000 = 3.0102
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8842
100%|██████████| 157/157 [00:03<00:00, 51.48it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2310
	Time_Epoch/train_phase/train_stream/Task000 = 3.0501
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9058
100%|██████████| 157/157 [00:03<00:00, 48.07it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1956
	Time_Epoch/train_phase/train_stream/Task000 = 3.2657
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9216
100%|██████████| 157/157 [00:02<00:00, 5

RuntimeError: Tensors must have same number of dimensions: got 4 and 2

In [ ]:
from collections import defaultdict
import random


In [ ]:
class EpisodicMemoryBuffer:
    def __init__(self, device, max_exemplars_per_class):
        self.device = device
        self.max_exemplars_per_class = max_exemplars_per_class
        self.memory = defaultdict(list)

    def add_samples(self, x, y):
        for xi, yi in zip(x, y):
            cls = yi.item()
            if len(self.memory[cls]) < self.max_exemplars_per_class:
                self.memory[cls].append((xi.detach().cpu(), yi.detach().cpu()))
            else:
                idx = torch.randint(0, self.max_exemplars_per_class, (1,)).item()
                self.memory[cls][idx] = (xi.detach().cpu(), yi.detach().cpu())

    def sample_batch(self, batch_size=32):
        if len(self.memory) == 0:
            return None, None

        class_samples = []
        num_classes = len(self.memory)
        if num_classes == 0:
            return None, None

        samples_per_class = max(1, batch_size // num_classes)

        for cls, items in self.memory.items():
            if len(items) == 0:
                continue
            n = min(samples_per_class, len(items))
            class_samples.extend(random.sample(items, n))

        if not class_samples:
            return None, None

        samples = random.sample(class_samples, min(batch_size, len(class_samples)))
        xs, ys = zip(*samples)
        return torch.stack(xs).to(self.device), torch.stack(ys).to(self.device)


In [ ]:
class ClusterAwareMinimalReplayCL(SupervisedTemplate):
    def __init__(self, *, model, optimizer, device, lambda_intra, lambda_anchor,
                 lambda_logit, lambda_var, criterion=torch.nn.CrossEntropyLoss(),
                 max_exemplars_per_class=30, use_replay=True, **kwargs):
        super().__init__(model=model, optimizer=optimizer, criterion=criterion, device=device, **kwargs)

        self.lambda_intra = lambda_intra
        self.lambda_anchor = lambda_anchor
        self.lambda_logit = lambda_logit
        self.lambda_var = lambda_var
        self.device = device

        self.prev_centroids = {}
        self.prev_variances = {}
        self.model_old = None

        self.replay_buffer = EpisodicMemoryBuffer(device, max_exemplars_per_class)
        self.use_replay = use_replay

    def forward(self):
        x = self.mb_x
        y = self.mb_y

        self.clean_x = x
        self.clean_y = y
        replay_mask = torch.zeros_like(y, dtype=torch.bool)

        if self.model.training and self.use_replay:
            rx, ry = self.replay_buffer.sample_batch(batch_size=x.size(0))
            if rx is not None:
                x = torch.cat([x, rx], dim=0)
                y = torch.cat([y, ry], dim=0)
                replay_mask = torch.cat([torch.zeros_like(self.mb_y, dtype=torch.bool), torch.ones_like(ry, dtype=torch.bool)], dim=0)

        self.replay_mask = replay_mask
        self.current_x = x
        self.current_y = y
        self.current_feats, self.logits = self.model(x, return_feats=True)
        return self.logits

    def criterion(self):
        feats = F.normalize(self.current_feats, p=2, dim=1)
        targets = self.current_y

        current_logits = self.logits[~self.replay_mask]
        current_targets = targets[~self.replay_mask]
        replay_logits = self.logits[self.replay_mask]
        replay_targets = targets[self.replay_mask]

        ce_current = self._criterion(current_logits, current_targets)
        ce_replay = self._criterion(replay_logits, replay_targets) if len(replay_targets) > 0 else 0.0
        ce_loss = ce_current + 0.5 * ce_replay

         # Feature regularization (L2 distance between feats and detached copy)
        feat_reg = 0.0
        if hasattr(self, 'prev_feats') and self.prev_feats is not None:
            feat_reg = F.mse_loss(self.current_feats, self.prev_feats)


        curr_centroids, curr_variances = {}, {}
        for cls in torch.unique(targets):
            mask = targets == cls
            feats_cls = feats[mask]
            curr_centroids[cls.item()] = feats_cls.mean(0)
            curr_variances[cls.item()] = feats_cls.var(0, unbiased=False)

        inter_cluster_loss = 0.0
        cls_list = list(curr_centroids.keys())
        for i in range(len(cls_list)):
            for j in range(i + 1, len(cls_list)):
                inter_cluster_loss += F.mse_loss(curr_centroids[cls_list[i]], curr_centroids[cls_list[j]])
        if len(cls_list) > 1:
            inter_cluster_loss /= len(cls_list) * (len(cls_list) - 1) / 2

        anchor_loss = 0.0
        for cls, prev_mu in self.prev_centroids.items():
            if cls in curr_centroids:
                anchor_loss += F.mse_loss(curr_centroids[cls], prev_mu.to(feats.device))
        if self.prev_centroids:
            anchor_loss /= len(self.prev_centroids)

        var_loss = 0.0
        for cls, prev_var in self.prev_variances.items():
            if cls in curr_variances:
                var_loss += F.mse_loss(curr_variances[cls], prev_var.to(feats.device))
        if self.prev_variances:
            var_loss /= len(self.prev_variances)

        logit_loss = 0.0
        if self.model_old is not None and self.replay_mask.any():
          with torch.no_grad():
              old_replay_logits = self.model_old(self.current_x[self.replay_mask])
          logit_loss = F.mse_loss(self.logits[self.replay_mask], old_replay_logits)

        softmax = F.softmax(self.logits[~self.replay_mask], dim=1)
        entropy = -torch.sum(softmax * torch.log(softmax + 1e-7), dim=1).mean()

        return (  ce_loss + 0.1 * feat_reg  + 0.2* entropy
                  + self.lambda_intra * inter_cluster_loss
                  + self.lambda_anchor * anchor_loss
                  + self.lambda_logit * logit_loss
                  + self.lambda_var * var_loss )



    def _after_training_exp(self, **kwargs):
        self.model.eval()
        dl = DataLoader(self.adapted_dataset, batch_size=128, shuffle=True)
        centroids, variances = defaultdict(list), defaultdict(list)
        with torch.no_grad():
            for x, y, *_ in dl:
                x, y = x.to(self.device), y.to(self.device)
                feats, _ = self.model(x, return_feats=True)
                feats = F.normalize(feats, p=2, dim=1)
                for cls in torch.unique(y):
                    mask = y == cls
                    if mask.any():
                        feats_cls = feats[mask]
                        centroids[cls.item()].append(feats_cls.mean(0))
                        variances[cls.item()].append(feats_cls.var(0, unbiased=False))
                self.replay_buffer.add_samples(x, y)

        for cls in centroids:
            self.prev_centroids[cls] = torch.stack(centroids[cls]).mean(0).detach().cpu()
            self.prev_variances[cls] = torch.stack(variances[cls]).mean(0).detach().cpu()

        self.model_old = self.model.__class__()
        self.model_old.load_state_dict(self.model.state_dict())
        self.model_old.to(self.device)
        self.model_old.eval()
        for p in self.model_old.parameters():
            p.requires_grad = False

        self.model.train()
        super()._after_training_exp(**kwargs)


In [ ]:
model = SimpleCNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=False, experience=True, stream=True),
    loggers=[InteractiveLogger()]
)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # ✅ correct


In [ ]:
# train & eval
strategy = ClusterAwareMinimalReplayCL(
    model=model,
    optimizer=optimizer,
    criterion=torch.nn.CrossEntropyLoss(),
    train_mb_size=64,
    train_epochs=5,
    eval_mb_size=128,
    device=device,
    evaluator=eval_plugin,
    lambda_intra= 0.05,
    max_exemplars_per_class= 100,
    lambda_logit= 0.2,
    lambda_anchor = 0.1,
    lambda_var = 0

)


for experience in benchmark_class_il.train_stream:
    current_classes = list(set(experience.dataset.targets))
    print(f"Training on classes: {current_classes}")
    print("Training on experience", experience.current_experience)
#//strategy.use_replay = True
    print("Replay active:", strategy.use_replay)

    strategy.train(experience)

    print("Evaluating on all seen tasks...")
    print("Evaluating... Replay should be OFF")
    print("Replay active:", strategy.use_replay)
    #strategy.use_replay = False

    strategy.eval(benchmark_class_il.test_stream[: experience.current_experience + 1])

Training on classes: [0, 1]
Training on experience 0
Replay active: True
-- >> Start of training phase << --
100%|██████████| 157/157 [00:03<00:00, 48.07it/s]
Epoch 0 ended.
100%|██████████| 157/157 [00:03<00:00, 48.51it/s]
Epoch 1 ended.
100%|██████████| 157/157 [00:03<00:00, 47.22it/s]
Epoch 2 ended.
100%|██████████| 157/157 [00:03<00:00, 49.74it/s]
Epoch 3 ended.
100%|██████████| 157/157 [00:03<00:00, 43.28it/s]
Epoch 4 ended.
-- >> End of training phase << --
Evaluating on all seen tasks...
Evaluating... Replay should be OFF
Replay active: True
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 16/16 [00:00<00:00, 46.96it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9485
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9485
Training on classes: [2, 3]
Training on experience 1
Replay active: True
-- >> Start of training ph

# Using resnet18

In [ ]:

class EpisodicMemoryBuffer:
    def __init__(self,device, max_exemplars_per_class=30):
        self.device = device
        self.max_exemplars_per_class = max_exemplars_per_class
        self.memory = defaultdict(list)

    def add_samples(self, x, y):
        for xi, yi in zip(x, y):
            cls = yi.item()
            if len(self.memory[cls]) < self.max_exemplars_per_class:
                self.memory[cls].append((xi.detach().cpu(), yi.detach().cpu()))
            else:
                idx = torch.randint(0, self.max_exemplars_per_class, (1,)).item()
                self.memory[cls][idx] = (xi.detach().cpu(), yi.detach().cpu())

    def sample_batch(self, batch_size=32):
        if len(self.memory) == 0:
            return None, None

        class_samples = []
        num_classes = len(self.memory)
        if num_classes == 0:
            return None, None

        samples_per_class = max(1, batch_size // num_classes)

        for cls, items in self.memory.items():
            if len(items) == 0:
                continue
            n = min(samples_per_class, len(items))
            class_samples.extend(random.sample(items, n))

        if not class_samples:
            return None, None

        samples = random.sample(class_samples, min(batch_size, len(class_samples)))
        xs, ys = zip(*samples)
        return torch.stack(xs).to(self.device), torch.stack(ys).to(self.device)

from copy import deepcopy

class ClusterAwareMinimalReplayCL(SupervisedTemplate):
    def __init__(self, *, model, optimizer, device, lambda_intra=1.0, lambda_anchor=0.1,
                 lambda_logit=0.05, lambda_var=0.0, criterion=torch.nn.CrossEntropyLoss(),
                 max_exemplars_per_class=30, use_replay=True, **kwargs):
        super().__init__(model=model, optimizer=optimizer, criterion=criterion, device=device, **kwargs)

        self.lambda_intra = lambda_intra
        self.lambda_anchor = lambda_anchor
        self.lambda_logit = lambda_logit
        self.lambda_var = lambda_var
        self.device = device

        self.prev_centroids = {}
        self.prev_variances = {}
        self.model_old = None
        self.old_centroids = {}  # task_id → centroid map
        self.task_accuracies = defaultdict(list)
        self.max_accuracies = defaultdict(float)

        self.replay_buffer = EpisodicMemoryBuffer(device, max_exemplars_per_class)
        self.use_replay = use_replay

        self.experience = None

    def criterion(self):
        feats = F.normalize(self.prev_feats, p=2, dim=1)
        targets = self.current_y

        current_logits = self.logits[~self.replay_mask]
        current_targets = targets[~self.replay_mask]
        replay_logits = self.logits[self.replay_mask]
        replay_targets = targets[self.replay_mask]

        ce_current = self._criterion(current_logits, current_targets)
        if len(replay_targets) > 0:
            # Dynamic weighting: prioritize forgotten or older task classes
            weights = torch.ones_like(replay_targets, dtype=torch.float)
            for i, y in enumerate(replay_targets):
                task_id = y.item() // 2  # assuming class group size 2 (SplitCIFAR10)
                if self.task_accuracies[task_id]:
                    recent_acc = self.task_accuracies[task_id][-1]
                    weights[i] = 1.5 if recent_acc < 0.5 else 1.0
            ce_replay = F.cross_entropy(replay_logits, replay_targets, reduction='none')
            ce_replay = (ce_replay * weights.to(replay_logits.device)).mean()
        else:
            ce_replay = 0.0
        ce_loss = ce_current + 0.5 * ce_replay

        # Feature regularization (L2 distance between feats and detached copy)
        feat_reg = 0.0
        if hasattr(self, 'prev_feats') and self.prev_feats is not None:
            feat_reg = F.mse_loss(self.prev_feats, self.prev_feats.detach())

        # Centroids and variance terms
        curr_centroids, curr_variances = {}, {}
        for cls in torch.unique(targets):
            mask = targets == cls
            feats_cls = feats[mask]
            curr_centroids[cls.item()] = feats_cls.mean(0)
            curr_variances[cls.item()] = feats_cls.var(0, unbiased=False)

        inter_cluster_loss = 0.0
        cls_list = list(curr_centroids.keys())
        for i in range(len(cls_list)):
            for j in range(i + 1, len(cls_list)):
                inter_cluster_loss += F.mse_loss(curr_centroids[cls_list[i]], curr_centroids[cls_list[j]])
        if len(cls_list) > 1:
            inter_cluster_loss /= len(cls_list) * (len(cls_list) - 1) / 2

        anchor_loss = 0.0
        for cls, prev_mu in self.prev_centroids.items():
            if cls in curr_centroids:
                anchor_loss += F.mse_loss(curr_centroids[cls], prev_mu.to(feats.device))
        if self.prev_centroids:
            anchor_loss /= len(self.prev_centroids)

        var_loss = 0.0
        for cls, prev_var in self.prev_variances.items():
            if cls in curr_variances:
                var_loss += F.mse_loss(curr_variances[cls], prev_var.to(feats.device))
        if self.prev_variances:
            var_loss /= len(self.prev_variances)

        logit_loss = 0.0
        if self.model_old is not None and self.replay_mask.any():
            with torch.no_grad():
                old_logits = self.model_old(self.current_x[self.replay_mask])
            logit_loss = F.mse_loss(self.logits[self.replay_mask], old_logits)

        return ce_loss + self.lambda_intra * inter_cluster_loss + \
               self.lambda_anchor * anchor_loss + \
               self.lambda_logit * logit_loss + \
               self.lambda_var * var_loss + 0.1 * feat_reg

    def forward(self):
        x = self.mb_x
        y = self.mb_y

        self.clean_x = x
        self.clean_y = y
        replay_mask = torch.zeros_like(y, dtype=torch.bool)

        if self.model.training and self.use_replay:
            rx, ry = self.replay_buffer.sample_batch(batch_size=x.size(0))
            if rx is not None:
                x = torch.cat([x, rx], dim=0)
                y = torch.cat([y, ry], dim=0)
                replay_mask = torch.cat([
                    torch.zeros_like(self.mb_y, dtype=torch.bool),
                    torch.ones_like(ry, dtype=torch.bool)
                ], dim=0)

        self.replay_mask = replay_mask
        self.current_x = x
        self.current_y = y
        if hasattr(self.model, 'forward_features'):
            feats = self.model.forward_features(x)
        else:
            feats = self.model.avgpool(
                self.model.layer4(
                    self.model.layer3(
                        self.model.layer2(
                            self.model.layer1(
                                self.model.relu(self.model.bn1(self.model.conv1(x)))
                            )
                        )
                    )
                )
            )
            feats = torch.flatten(feats, 1)
        self.prev_feats = feats
        self.logits = self.model.fc(feats)
        return self.logits

    def _after_training_exp(self, **kwargs):
        self.model.eval()
        dl = DataLoader(self.adapted_dataset, batch_size=128, shuffle=True)
        centroids, variances = defaultdict(list), defaultdict(list)
        with torch.no_grad():
            for x, y, *_ in dl:
                x, y = x.to(self.device), y.to(self.device)
                feats = self.model.forward_features(x) if hasattr(self.model, 'forward_features') else torch.flatten(self.model.avgpool(
                    self.model.layer4(
                        self.model.layer3(
                            self.model.layer2(
                                self.model.layer1(
                                    self.model.relu(self.model.bn1(self.model.conv1(x)))
                                )
                            )
                        )
                    )), 1)
                for cls in torch.unique(y):
                    mask = y == cls
                    if mask.any():
                        feats_cls = feats[mask]
                        centroids[cls.item()].append(feats_cls.mean(0))
                        variances[cls.item()].append(feats_cls.var(0, unbiased=False))
                self.replay_buffer.add_samples(x, y)

        task_id = self.experience.current_experience
        self.old_centroids[task_id] = centroids  # Store for future reference

        for cls in centroids:
            self.prev_centroids[cls] = torch.stack(centroids[cls]).mean(0).detach().cpu()
            self.prev_variances[cls] = torch.stack(variances[cls]).mean(0).detach().cpu()

        self.model_old = deepcopy(self.model)
        self.model_old.to(self.device)
        self.model_old.eval()
        for p in self.model_old.parameters():
            p.requires_grad = False

        self.model.train()
        super()._after_training_exp(**kwargs)


In [ ]:
from torchvision.models import resnet18

model = resnet18(pretrained=False, num_classes=10)
model.fc = torch.nn.Linear(model.fc.in_features, 10)
model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = torch.nn.Identity()  # remove downsampling for CIFAR
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# train & eval
strategy = ClusterAwareMinimalReplayCL(
    model=model,
    optimizer=optimizer,
    criterion=torch.nn.CrossEntropyLoss(),
    train_mb_size=64,
    train_epochs=5,
    eval_mb_size=128,
    device=device,
    evaluator=eval_plugin,
    lambda_intra= 1,
    max_exemplars_per_class= 30,
    lambda_logit= 1,
    lambda_anchor = 0.5,
    lambda_var = 0

)


for experience in benchmark_class_il.train_stream:
    current_classes = list(set(experience.dataset.targets))
    print(f"Training on classes: {current_classes}")
    print("Training on experience", experience.current_experience)
#//strategy.use_replay = True

    strategy.train(experience)

    print("Evaluating on all seen tasks...")
    print("Evaluating... Replay should be OFF")

    #strategy.use_replay = False

    strategy.eval(benchmark_class_il.test_stream[: experience.current_experience + 1])

    results = strategy.evaluator.get_last_metrics()

    # Update task accuracy
    for key, value in results.items():
        if "Top1_Acc_Exp" in key:
            task_id = int(key.split("Exp")[-1])
            strategy.task_accuracies[task_id].append(value)
            strategy.max_accuracies[task_id] = max(
                strategy.max_accuracies[task_id], value)


Training on classes: [0, 1]
Training on experience 0
-- >> Start of training phase << --
100%|██████████| 157/157 [00:37<00:00,  4.14it/s]
Epoch 0 ended.
100%|██████████| 157/157 [00:36<00:00,  4.25it/s]
Epoch 1 ended.
100%|██████████| 157/157 [00:36<00:00,  4.29it/s]
Epoch 2 ended.
100%|██████████| 157/157 [00:38<00:00,  4.05it/s]
Epoch 3 ended.
100%|██████████| 157/157 [00:41<00:00,  3.82it/s]
Epoch 4 ended.
-- >> End of training phase << --
Evaluating on all seen tasks...
Evaluating... Replay should be OFF
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 16/16 [00:02<00:00,  6.35it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9880
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9880
Training on classes: [2, 3]
Training on experience 1
-- >> Start of training phase << --
100%|██████████| 157/157 [01:23<00:00,  1.89it/s]


In [ ]:
class EpisodicMemoryBuffer:
    def __init__(self, device,  max_exemplars_per_class=30):
        self.device = device
        self.max_exemplars_per_class = max_exemplars_per_class
        self.memory = defaultdict(list)

    def add_samples(self, x, y):
        for xi, yi in zip(x, y):
            cls = yi.item()
            if len(self.memory[cls]) < self.max_exemplars_per_class:
                self.memory[cls].append((xi.detach().cpu(), yi.detach().cpu()))
            else:
                idx = torch.randint(0, self.max_exemplars_per_class, (1,)).item()
                self.memory[cls][idx] = (xi.detach().cpu(), yi.detach().cpu())

    def sample_batch(self, batch_size=32):
        if len(self.memory) == 0:
            return None, None

        class_samples = []
        num_classes = len(self.memory)
        if num_classes == 0:
            return None, None

        samples_per_class = max(1, batch_size // num_classes)

        for cls, items in self.memory.items():
            if len(items) == 0:
                continue
            n = min(samples_per_class, len(items))
            class_samples.extend(random.sample(items, n))

        if not class_samples:
            return None, None

        samples = random.sample(class_samples, min(batch_size, len(class_samples)))
        xs, ys = zip(*samples)
        return torch.stack(xs).to(self.device), torch.stack(ys).to(self.device)

from copy import deepcopy

class ClusterAwareMinimalReplayCL(SupervisedTemplate):
    def __init__(self, *, model, optimizer, device, lambda_intra=1.0, lambda_anchor=0.1,
                 lambda_logit=0.05, lambda_var=0.0, criterion=torch.nn.CrossEntropyLoss(),
                 max_exemplars_per_class=30, use_replay=True, **kwargs):
        super().__init__(model=model, optimizer=optimizer, criterion=criterion, device=device, **kwargs)

        self.lambda_intra = lambda_intra
        self.lambda_anchor = lambda_anchor
        self.lambda_logit = lambda_logit
        self.lambda_var = lambda_var
        self.device = device

        self.prev_centroids = {}
        self.prev_variances = {}
        self.model_old = None
        self.old_centroids = {}  # task_id → centroid map
        self.task_accuracies = defaultdict(list)
        self.max_accuracies = defaultdict(float)

        self.replay_buffer = EpisodicMemoryBuffer(device, max_exemplars_per_class)
        self.use_replay = use_replay

        self.experience = None

        # -------------------------
    # 🎯 Accuracy + Replay Debug Logic
    # -------------------------

    def update_task_accuracy(self, results):
        for key, value in results.items():
            if "Top1_Acc_Exp" in key:
                task_id = int(key.split("Exp")[-1])
                self.task_accuracies[task_id].append(value)
                self.max_accuracies[task_id] = max(self.max_accuracies[task_id], value)

    # -------------------------
    # 🔍 Loss Function (with dynamic replay weight)
    # -------------------------

    def criterion(self):
        feats = F.normalize(self.prev_feats, p=2, dim=1)
        targets = self.current_y

        current_logits = self.logits[~self.replay_mask]
        current_targets = targets[~self.replay_mask]
        replay_logits = self.logits[self.replay_mask]
        replay_targets = targets[self.replay_mask]

        ce_current = self._criterion(current_logits, current_targets)
        # if len(replay_targets) > 0:
        #     # Dynamic weighting: prioritize forgotten or older task classes
        #     weights = torch.ones_like(replay_targets, dtype=torch.float)
        #     for i, y in enumerate(replay_targets):
        #         task_id = y.item() // 2  # assuming class group size 2 (SplitCIFAR10)
        #         if self.task_accuracies[task_id]:
        #             recent_acc = self.task_accuracies[task_id][-1]
        #             weights[i] = 1.5 if recent_acc < 0.5 else 1.0
        #     ce_replay = F.cross_entropy(replay_logits, replay_targets, reduction='none')
        #     ce_replay = (ce_replay * weights.to(replay_logits.device)).mean()
        # else:
        #     ce_replay = 0.0
        ce_replay = self._criterion(replay_logits, replay_targets) if len(replay_targets) > 0 else 0.0
        ce_loss = ce_current + 0.5 * ce_replay

        #ce_loss = ce_current + 0.5 * ce_replay

        # Feature regularization (L2 distance between feats and detached copy)
        feat_reg = 0.0
        if hasattr(self, 'prev_feats') and self.prev_feats is not None:
            feat_reg = F.mse_loss(self.prev_feats, self.prev_feats.detach())

        # Centroids and variance terms
        curr_centroids, curr_variances = {}, {}
        for cls in torch.unique(targets):
            mask = targets == cls
            feats_cls = feats[mask]
            curr_centroids[cls.item()] = feats_cls.mean(0)
            curr_variances[cls.item()] = feats_cls.var(0, unbiased=False)

        inter_cluster_loss = 0.0
        cls_list = list(curr_centroids.keys())
        for i in range(len(cls_list)):
            for j in range(i + 1, len(cls_list)):
                inter_cluster_loss += F.mse_loss(curr_centroids[cls_list[i]], curr_centroids[cls_list[j]])
        if len(cls_list) > 1:
            inter_cluster_loss /= len(cls_list) * (len(cls_list) - 1) / 2

        anchor_loss = 0.0
        for cls, prev_mu in self.prev_centroids.items():
            if cls in curr_centroids:
                anchor_loss += F.mse_loss(curr_centroids[cls], prev_mu.to(feats.device))
        if self.prev_centroids:
            anchor_loss /= len(self.prev_centroids)

        var_loss = 0.0
        for cls, prev_var in self.prev_variances.items():
            if cls in curr_variances:
                var_loss += F.mse_loss(curr_variances[cls], prev_var.to(feats.device))
        if self.prev_variances:
            var_loss /= len(self.prev_variances)

        logit_loss = 0.0
        if self.model_old is not None and self.replay_mask.any():
            with torch.no_grad():
                old_logits = self.model_old(self.current_x[self.replay_mask])
            logit_loss = F.mse_loss(self.logits[self.replay_mask], old_logits)

        return ce_loss + self.lambda_intra * inter_cluster_loss + \
               self.lambda_anchor * anchor_loss + \
               self.lambda_logit * logit_loss + \
               self.lambda_var * var_loss + 0.1 * feat_reg

    def forward(self):
        x = self.mb_x
        y = self.mb_y

        self.clean_x = x
        self.clean_y = y
        replay_mask = torch.zeros_like(y, dtype=torch.bool)

        if self.model.training and self.use_replay:
            rx, ry = self.replay_buffer.sample_batch(batch_size=x.size(0))
            if rx is not None:
                x = torch.cat([x, rx], dim=0)
                y = torch.cat([y, ry], dim=0)
                replay_mask = torch.cat([
                    torch.zeros_like(self.mb_y, dtype=torch.bool),
                    torch.ones_like(ry, dtype=torch.bool)
                ], dim=0)

        self.replay_mask = replay_mask
        self.current_x = x
        self.current_y = y
        if hasattr(self.model, 'forward_features'):
            feats = self.model.forward_features(x)
        else:
            feats = self.model.avgpool(
                self.model.layer4(
                    self.model.layer3(
                        self.model.layer2(
                            self.model.layer1(
                                self.model.relu(self.model.bn1(self.model.conv1(x)))
                            )
                        )
                    )
                )
            )
            feats = torch.flatten(feats, 1)
        self.prev_feats = feats
        self.logits = self.model.fc(feats)
        return self.logits

    def _after_training_exp(self, **kwargs):
        self.model.eval()
        dl = DataLoader(self.adapted_dataset, batch_size=128, shuffle=True)
        centroids, variances = defaultdict(list), defaultdict(list)
        with torch.no_grad():
            for x, y, *_ in dl:
                x, y = x.to(self.device), y.to(self.device)
                feats = self.model.forward_features(x) if hasattr(self.model, 'forward_features') else torch.flatten(self.model.avgpool(
                    self.model.layer4(
                        self.model.layer3(
                            self.model.layer2(
                                self.model.layer1(
                                    self.model.relu(self.model.bn1(self.model.conv1(x)))
                                )
                            )
                        )
                    )), 1)
                for cls in torch.unique(y):
                    mask = y == cls
                    if mask.any():
                        feats_cls = feats[mask]
                        centroids[cls.item()].append(feats_cls.mean(0))
                        variances[cls.item()].append(feats_cls.var(0, unbiased=False))
                self.replay_buffer.add_samples(x, y)

        task_id = self.experience.current_experience
        self.old_centroids[task_id] = centroids  # Store for future reference

        for cls in centroids:
            self.prev_centroids[cls] = torch.stack(centroids[cls]).mean(0).detach().cpu()
            self.prev_variances[cls] = torch.stack(variances[cls]).mean(0).detach().cpu()

        self.model_old = resnet18(pretrained=False, num_classes=self.model.fc.out_features)
        self.model_old.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model_old.maxpool = torch.nn.Identity()
        self.model_old.load_state_dict(self.model.state_dict())
        self.model_old.to(self.device)
        self.model_old.eval()
        for p in self.model_old.parameters():
            p.requires_grad = False

        self.model.train()
        super()._after_training_exp(**kwargs)


In [ ]:
# train & eval
strategy = ClusterAwareMinimalReplayCL(
    model=model,
    optimizer=optimizer,
    criterion=torch.nn.CrossEntropyLoss(),
    train_mb_size=64,
    train_epochs=5,
    eval_mb_size=128,
    device=device,
    evaluator=eval_plugin,
    lambda_intra= 1,
    max_exemplars_per_class= 30,
    lambda_logit= 1,
    lambda_anchor = 0.5,
    lambda_var = 0

)


for experience in benchmark_class_il.train_stream:
    current_classes = list(set(experience.dataset.targets))
    print(f"Training on classes: {current_classes}")
    print("Training on experience", experience.current_experience)
#//strategy.use_replay = True

    strategy.train(experience)

    print("Evaluating on all seen tasks...")
    print("Evaluating... Replay should be OFF")

    #strategy.use_replay = False

    strategy.eval(benchmark_class_il.test_stream[: experience.current_experience + 1])

    #results = strategy.evaluator.get_last_metrics()

Training on classes: [0, 1]
Training on experience 0
-- >> Start of training phase << --
100%|██████████| 157/157 [00:34<00:00,  4.51it/s]
Epoch 0 ended.
100%|██████████| 157/157 [00:39<00:00,  4.01it/s]
Epoch 1 ended.
100%|██████████| 157/157 [00:36<00:00,  4.35it/s]
Epoch 2 ended.
100%|██████████| 157/157 [00:39<00:00,  3.97it/s]
Epoch 3 ended.
100%|██████████| 157/157 [00:39<00:00,  4.00it/s]
Epoch 4 ended.


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


-- >> End of training phase << --
Evaluating on all seen tasks...
Evaluating... Replay should be OFF
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 16/16 [00:02<00:00,  7.39it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9855
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9855
Training on classes: [2, 3]
Training on experience 1
-- >> Start of training phase << --
100%|██████████| 157/157 [01:11<00:00,  2.18it/s]
Epoch 0 ended.
100%|██████████| 157/157 [01:15<00:00,  2.08it/s]
Epoch 1 ended.
100%|██████████| 157/157 [01:19<00:00,  1.99it/s]
Epoch 2 ended.
100%|██████████| 157/157 [01:07<00:00,  2.32it/s]
Epoch 3 ended.
100%|██████████| 157/157 [01:04<00:00,  2.44it/s]
Epoch 4 ended.
-- >> End of training phase << --
Evaluating on all seen tasks...
Evaluating... Replay should be OFF
-- >> Start of eval phase << --
-- Sta

KeyboardInterrupt: 

In [ ]:
from avalanche.training.supervised import Replay

strategy = Naive(
    model=model,
    optimizer=optimizer,
    criterion=torch.nn.CrossEntropyLoss(),
    train_mb_size=64,
    train_epochs=5,
    eval_mb_size=128,
    device=device,
)

for experience in benchmark_class_il.train_stream:
    current_classes = list(set(experience.dataset.targets))
    print(f"Training on classes: {current_classes}")
    print("Training on experience", experience.current_experience)

    strategy.train(experience)

    print("Evaluating on all seen tasks...")

    strategy.eval(benchmark_class_il.test_stream[: experience.current_experience + 1])

Training on classes: [0, 1]
Training on experience 0
-- >> Start of training phase << --
 22%|██▏       | 34/157 [00:06<00:21,  5.72it/s]

KeyboardInterrupt: 

In [ ]:
from avalanche.training.supervised import Replay

strategy = Replay(
    model=model,
    optimizer=optimizer,
    criterion=torch.nn.CrossEntropyLoss(),
    mem_size=200,
    train_mb_size=64,
    train_epochs=5,
    eval_mb_size=128,
    device=device,
)

for experience in benchmark_class_il.train_stream:
    current_classes = list(set(experience.dataset.targets))
    print(f"Training on classes: {current_classes}")
    print("Training on experience", experience.current_experience)

    strategy.train(experience)

    print("Evaluating on all seen tasks...")

    strategy.eval(benchmark_class_il.test_stream[: experience.current_experience + 1])

In [ ]:
# EWC baseline ------------------------------------------------------------
from avalanche.training.supervised import EWC

strategy = EWC(
    model=model,
    optimizer=optimizer,
    criterion=torch.nn.CrossEntropyLoss(),
    ewc_lambda=0.4,  # importance of penalty
    mode='separate',  # one FIM per task
    train_mb_size=64,
    train_epochs=5,
    eval_mb_size=128,
    device=device,
)

for experience in benchmark_class_il.train_stream:
    current_classes = list(set(experience.dataset.targets))
    print(f"Training on classes: {current_classes}")
    print("Training on experience", experience.current_experience)

    strategy.train(experience)

    print("Evaluating on all seen tasks...")

    strategy.eval(benchmark_class_il.test_stream[: experience.current_experience + 1])

In [ ]:
class ResNet18(nn.Module):
    """A ResNet-18 model adapted for feature extraction."""
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()
        base_model = resnet18(weights=None)
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        self.classifier = nn.Linear(base_model.fc.in_features, num_classes)

    def forward(self, x):
        f = self.features(x)
        f = f.view(f.size(0), -1)
        y = self.classifier(f)
        return y

# =============================================================================
# 2. DATA HANDLING & REPLAY BUFFER
# =============================================================================

class TaskDataset(Dataset):
    """A custom dataset to provide data for a specific task (a set of classes)."""
    def __init__(self, dataset, task_classes):
        self.dataset = dataset
        self.task_classes = task_classes
        self.indices = []
        targets = np.array(self.dataset.targets)
        for cls in task_classes:
            self.indices.extend(np.where(targets == cls)[0])

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

class ReplayBuffer(Dataset):
    """A tiny memory buffer to store exemplars from past tasks."""
    def __init__(self, exemplars_per_class=10):
        self.exemplars_per_class = exemplars_per_class
        self.memory = {}

    def add_exemplars(self, dataset, task_classes):
        """Adds exemplars for the given classes to the memory."""
        for cls in task_classes:
            cls_dataset = TaskDataset(dataset, [cls])
            self.memory[cls] = [cls_dataset[i] for i in range(len(cls_dataset))]
            if len(self.memory[cls]) > self.exemplars_per_class:
                self.memory[cls] = random.sample(self.memory[cls], self.exemplars_per_class)

    def __len__(self):
        return sum(len(samples) for samples in self.memory.values())

    def __getitem__(self, idx):
        for cls, samples in self.memory.items():
            if idx < len(samples):
                return samples[idx]
            idx -= len(samples)
        raise IndexError("Index out of range")

# =============================================================================
# 3. CUSTOM LOSS FUNCTION (FITNESS FUNCTION)
# =============================================================================

class InterClusterLoss(nn.Module):
    """
    This loss acts as a "fitness function" to maximize the inter-cluster distance.
    It pushes the feature clusters of new classes away from the centroids of old classes.
    """
    def __init__(self, lambda_reg=1.0):
        super(InterClusterLoss, self).__init__()
        self.lambda_reg = lambda_reg
        self.class_centroids = {}

    def forward(self, features, targets):
        """Calculates the fitness by maximizing Euclidean distance between NORMALIZED features."""
        if not self.class_centroids:
            return torch.tensor(0.0, device=features.device)

        # *** CRITICAL FIX: Normalize features to have a unit length (L2 norm = 1) ***
        # This prevents the loss from exploding by stopping the model from making feature vectors infinitely long.
        features = F.normalize(features.view(features.size(0), -1), p=2, dim=1)

        loss = torch.tensor(0.0, device=features.device)
        count = 0
        current_classes = torch.unique(targets)

        for cls_idx in current_classes:
            class_features = features[targets == cls_idx]
            for old_cls_idx, old_centroid in self.class_centroids.items():
                if cls_idx.item() != old_cls_idx:
                    distance = torch.cdist(class_features, old_centroid.unsqueeze(0), p=2).mean()
                    loss += -distance
                    count += 1

        if count == 0:
            return torch.tensor(0.0, device=features.device)
        return self.lambda_reg * (loss / count)

    def update_centroids(self, model, dataset, task_classes, device):
        """Updates centroids for the new classes using NORMALIZED features."""
        model.eval()
        with torch.no_grad():
            for cls in task_classes:
                class_dataset = TaskDataset(dataset, [cls])
                loader = DataLoader(class_dataset, batch_size=64, shuffle=False)
                class_features = []
                for inputs, _ in loader:
                    inputs = inputs.to(device)
                    # *** CRITICAL FIX: Normalize features before averaging to create centroids ***
                    features = model.features(inputs)
                    normalized_features = F.normalize(features.view(features.size(0), -1), p=2, dim=1)
                    class_features.append(normalized_features)

                self.class_centroids[cls] = torch.cat(class_features, dim=0).mean(dim=0).detach()
        print(f"Updated centroids. Now tracking: {list(self.class_centroids.keys())}")

# =============================================================================
# 4. TRAINING & EVALUATION
# =============================================================================

def train(model, loader, optimizer, criterion_ce, criterion_inter_cluster, device):
    """Training loop for one epoch, now handles combined current and replay data."""
    model.train()
    total_loss = 0
    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        # We need the raw (non-normalized) features for the classifier
        features_raw = model.features(inputs)
        outputs = model.classifier(features_raw.view(features_raw.size(0), -1))

        loss_ce = criterion_ce(outputs, targets)
        # The inter-cluster loss will handle normalization internally
        loss_inter = criterion_inter_cluster(features_raw, targets)
        loss = loss_ce + loss_inter

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

def evaluate(model, loader, device):
    """Evaluation loop."""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    return 100 * correct / total

# =============================================================================
# 5. MAIN SCRIPT
# ====================================================

In [ ]:

    NUM_TASKS = 5
    CLASSES_PER_TASK = 2
    EPOCHS_PER_TASK = 10
    BATCH_SIZE = 32
    LEARNING_RATE = 0.001
    # *** CRITICAL FIX: Reduced lambda to a more reasonable starting value for a stabilized loss ***
    LAMBDA_REG = 10.0
    EXEMPLARS_PER_CLASS = 20
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {DEVICE}")

    # --- Data Preparation ---
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    class_order = np.arange(10)
    tasks = [class_order[i:i+CLASSES_PER_TASK] for i in range(0, 10, CLASSES_PER_TASK)]

    # --- Model and Loss Initialization ---
    device = torch.device(DEVICE)
    model = ResNet18(num_classes=10).to(device)
    criterion_ce = nn.CrossEntropyLoss()
    criterion_inter_cluster = InterClusterLoss(lambda_reg=LAMBDA_REG).to(device)
    replay_buffer = ReplayBuffer(exemplars_per_class=EXEMPLARS_PER_CLASS)

    # --- Continual Learning Loop ---
    for task_id, task_classes in enumerate(tasks):
        print(f"\n--- Training on Task {task_id+1}/{NUM_TASKS} (Classes: {task_classes}) ---")

        current_task_dataset = TaskDataset(train_dataset, task_classes)

        if len(replay_buffer) > 0:
            combined_dataset = ConcatDataset([current_task_dataset, replay_buffer])
            train_loader = DataLoader(combined_dataset, batch_size=BATCH_SIZE, shuffle=True)
        else:
            train_loader = DataLoader(current_task_dataset, batch_size=BATCH_SIZE, shuffle=True)

        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        for epoch in range(EPOCHS_PER_TASK):
            train_loss = train(model, train_loader, optimizer, criterion_ce, criterion_inter_cluster, device)
            print(f"Epoch {epoch+1}/{EPOCHS_PER_TASK}, Loss: {train_loss:.4f}")

        criterion_inter_cluster.update_centroids(model, train_dataset, task_classes, device)
        replay_buffer.add_exemplars(train_dataset, task_classes)
        print(f"Replay buffer size: {len(replay_buffer)} samples")

    # --- Final Evaluation ---
    print("\n--- Final Evaluation after all tasks ---")
    accuracies = []
    for task_id, task_classes in enumerate(tasks):
        test_task_dataset = TaskDataset(test_dataset, task_classes)
        test_loader = DataLoader(test_task_dataset, batch_size=BATCH_SIZE, shuffle=False)
        acc = evaluate(model, test_loader, device)
        print(f"Accuracy on Task {task_id+1} (Classes {task_classes}): {acc:.2f}%")
        accuracies.append(acc)

    average_accuracy = np.mean(accuracies)
    print(f"\nAverage Accuracy across all tasks: {average_accuracy:.2f}%")


Using device: cpu

--- Training on Task 1/5 (Classes: [0 1]) ---
Epoch 1/10, Loss: 0.3603
Epoch 2/10, Loss: 0.2057
Epoch 3/10, Loss: 0.1425
Epoch 4/10, Loss: 0.1222
Epoch 5/10, Loss: 0.1029
Epoch 6/10, Loss: 0.0840
Epoch 7/10, Loss: 0.0723
Epoch 8/10, Loss: 0.0588
Epoch 9/10, Loss: 0.0619
Epoch 10/10, Loss: 0.0465
Updated centroids. Now tracking: [np.int64(0), np.int64(1)]
Replay buffer size: 40 samples

--- Training on Task 2/5 (Classes: [2 3]) ---
Epoch 1/10, Loss: -11.4391
Epoch 2/10, Loss: -12.5402
Epoch 3/10, Loss: -12.5743
Epoch 4/10, Loss: -12.6792
Epoch 5/10, Loss: -12.7201
Epoch 6/10, Loss: -12.7619
Epoch 7/10, Loss: -12.8041
Epoch 8/10, Loss: -12.8292
Epoch 9/10, Loss: -12.8973
Epoch 10/10, Loss: -12.9597
Updated centroids. Now tracking: [np.int64(0), np.int64(1), np.int64(2), np.int64(3)]
Replay buffer size: 80 samples

--- Training on Task 3/5 (Classes: [4 5]) ---
Epoch 1/10, Loss: -8.4615
Epoch 2/10, Loss: -12.3687
Epoch 3/10, Loss: -12.5249
Epoch 4/10, Loss: -12.5977
Epoc